In [1]:
import sys
import tqdm
import time
import sklearn
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
import torch
import torch.nn.functional as F
from load_dataset import load
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from scipy import stats
from scipy.stats import rankdata
from sklearn import metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from operator import itemgetter
from torch.autograd import grad
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Load Dataset**

In [3]:
dataset = 'salary'
X_train, X_test, y_train, y_test = load(dataset)

**Parametric Model**

In [4]:
# size=500
# X_train = X_train[0:size]
# y_train = y_train[0:size]

X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function** (Log loss for logistic regression)

In [5]:
# clf = NeuralNetwork(input_size=X_train.shape[-1])
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression) or isinstance(clf, NeuralNetwork):
#     loss_func = lambda model, x, y_true: logistic_loss_torch(model(torch.FloatTensor(x)),\
#                                                              torch.FloatTensor([y_true])) +\
#     model.C*torch.sqrt(torch.sum(convert_grad_to_tensor(list(clf.parameters()))**2))
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch

**Influence of points computed using ground truth**

In [6]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test):
    clf.fit(X_train, y_train, verbose=True)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig, y_test, 0)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig, y_test, 0) - spd_0
        delta_spd.append(delta_spd_i)

    return delta_spd

**Compute Accuracy** 

In [7]:
def computeAccuracy(y_true, y_pred):
    return np.sum((y_pred>0.5) == y_true)/len(y_pred)

**First-order derivative of loss function at z with respect to model parameters**

In [8]:
def del_L_del_theta_i(model, x, y_true, retain_graph=False):
    loss = loss_func(model, x, y_true)
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(loss, w, create_graph=True, retain_graph=retain_graph)

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [9]:
def del_f_del_theta_i(model, x, retain_graph=False):
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(model(torch.FloatTensor(x)), w, retain_graph=retain_graph)

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [10]:
def hvp(y, w, v):
    ''' Multiply the Hessians of y and w by v.'''
    # First backprop
    first_grads = grad(y, w, retain_graph=True, create_graph=True)

    # Elementwise products
    elemwise_products = 0
    for grad_elem, v_elem in zip(convert_grad_to_tensor(first_grads), v):
        elemwise_products += torch.sum(grad_elem * v_elem)

    # Second backprop
    return_grads = grad(elemwise_products, w, create_graph=True)

    return return_grads

In [11]:
def hessian_one_point(model, x, y):
    x, y = torch.FloatTensor(x), torch.FloatTensor([y])
    loss = loss_func(model, x, y)
    params = [ p for p in model.parameters() if p.requires_grad ]
    first_grads = convert_grad_to_tensor(grad(loss, params, retain_graph=True, create_graph=True))
    hv = np.zeros((len(first_grads), len(first_grads)))
    for i in range(len(first_grads)):
        hv[i, :] = convert_grad_to_ndarray(grad(first_grads[i], params, create_graph=True)).ravel()
    return hv

In [12]:
# Compute multiplication of inverse hessian matrix and vector v
def s_test(model, xs, ys, v, hinv=None, damp=0.01, scale=25.0, r=-1, batch_size=-1, recursive=False, verbose=False):
    ''' Arguments:
        xs: list of data points
        ys: list of true labels corresponding to data points in xs
        damp: dampening factor
        scale: scaling factor
        r: number of iterations aka recursion depth
            should be enough so that the value stabilises.
        batch_size: number of instances in each batch in recursive approximation
        recursive: determine whether to recursively approximate hinv_v'''
    xs, ys = torch.FloatTensor(xs.copy()), torch.FloatTensor(ys.copy())
    n = len(xs)
    if recursive:
        hinv_v = copy.deepcopy(v)
        if verbose:
            print('Computing s_test...')
            tbar = tqdm.tqdm(total=r)
        if (batch_size == -1):  # default
            batch_size = 10
        if (r == -1):
            r = n // batch_size + 1
        sample = np.random.choice(range(n), r*batch_size, replace=True)
        for i in range(r):
            sample_idx = sample[i*batch_size:(i+1)*batch_size]
            x, y = xs[sample_idx], ys[sample_idx]
            loss = loss_func(model, x, y)
            params = [ p for p in model.parameters() if p.requires_grad ]
            hv = convert_grad_to_ndarray(hvp(loss, params, torch.FloatTensor(hinv_v)))
            # Recursively caclulate h_estimate
            hinv_v = v + (1 - damp) * hinv_v - hv / scale
            if verbose:
                tbar.update(1)
    else:
        if hinv is None:
            hinv = np.linalg.pinv(np.sum(hessian_all_points, axis=0))
        scale = 1.0
        hinv_v = np.matmul(hinv, v)

    return hinv_v / scale

**Metrics: Initial state**

In [13]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

# tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
# print("Initial TPR parity: ", tpr_parity_0)

# predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
# print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.3634165891756614
Initial loss:  0.23515541751875604
Initial accuracy:  0.8181818181818182


In [14]:
hessian_all_points = []
tbar = tqdm.tqdm(total=len(X_train))
total_time = 0
for i in range(len(X_train)):
    t0 = time.time()
    hessian_all_points.append(hessian_one_point(clf, X_train[i], y_train[i])/len(X_train))
    total_time += time.time()-t0
    tbar.update(1)

 41%|████▏     | 17/41 [00:00<00:00, 165.83it/s]

In [15]:
total_time

0.167924165725708

**Pre-compute: (1) Hessian (2) del_L_del_theta for each training data point**

In [16]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    del_L_del_theta.append(convert_grad_to_ndarray(del_L_del_theta_i(clf, X_train[i], y_train[i])))

*Select delta fairness function depending on selected metric*

In [17]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(clf, X_test_orig, X_test, dataset)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)

In [18]:
hinv = np.linalg.pinv(np.sum(hessian_all_points, axis=0))
hinv_v = s_test(clf, X_train, y_train, v1, hinv=hinv, verbose=False)

**First-order influence computation**

In [19]:
def first_order_influence(del_L_del_theta, hinv_v, n):
    infs = []
    for i in range(n):
        inf = -np.dot(del_L_del_theta[i].transpose(), hinv_v)
        inf *= -1/n
        infs.append(inf)
    return infs

In [20]:
def first_order_group_influence(U, del_L_del_theta):
    infs = []
    u = len(U)
    n = len(X_train)
    for i in range(u):
        idx = U[i]
        inf = -np.dot(del_L_del_theta[idx].transpose(), hinv)
        inf *= -1/n
        infs.append(inf)
    return np.sum(infs, axis=0)

**Second-order influence computation for a group of points in subset U**

In [21]:
def second_order_influence(model, X_train, y_train, U, del_L_del_theta, r=-1, verbose=False):
    u = len(U)
    s = len(X_train)
    p = u/s
    c1 = (1 - 2*p)/(s * (1-p)**2)
    c2 = 1/((s * (1-p))**2)
    num_params = len(del_L_del_theta[0])
    del_L_del_theta_sum = np.sum([del_L_del_theta[i] for i in U], axis=0)
    hinv_del_L_del_theta= s_test(model, X_train, y_train, del_L_del_theta_sum, hinv=hinv)
    hessian_U_hinv_del_L_del_theta = np.zeros((num_params,))
    for i in range(u):
        idx = U[i]
        x, y = torch.FloatTensor(X_train[idx]), torch.FloatTensor([y_train[idx]])
        loss = loss_func(model, x, y)
        params = [ p for p in model.parameters() if p.requires_grad ]
        hessian_U_hinv_del_L_del_theta += convert_grad_to_ndarray(hvp(loss, params, torch.FloatTensor(hinv_del_L_del_theta)))

    term1 = c1 * hinv_del_L_del_theta
    term2 = c2 * s_test(model, X_train, y_train, hessian_U_hinv_del_L_del_theta, hinv=hinv)
    sum_term = term1 + term2
    return sum_term

In [22]:
def second_order_group_influence(U, del_L_del_theta):
    u = len(U)
    s = len(X_train)
    p = u/s
    c1 = (1 - 2*p)/(s * (1-p)**2)
    c2 = 1/((s * (1-p))**2)
    num_params = len(del_L_del_theta[0])
    del_L_del_theta_sum = np.sum([del_L_del_theta[i] for i in U], axis=0)
    hinv_del_L_del_theta= np.matmul(hinv, del_L_del_theta_sum)
    hessian_U_hinv_del_L_del_theta = np.zeros((num_params,))
    for i in range(u):
        idx = U[i]
        hessian_U_hinv_del_L_del_theta += np.matmul(hessian_all_points[idx], hinv_del_L_del_theta)

    term1 = c1 * hinv_del_L_del_theta
    term2 = c2 * np.matmul(hinv, hessian_U_hinv_del_L_del_theta)
    sum_term = (term1 + term2*len(X_train))
    return sum_term

**First-order influence of each training data point**

In [23]:
infs_1 = first_order_influence(del_L_del_theta, hinv_v, len(X_train))

**Fairness: Ground-truth subset influence vs. computed subset influences: Coherent subset** 

(by coherent, we mean group of data points that share some properties)

***NOTE:*** The retraining of the clf would cause the change in model parameters and thus lead to the change of gradients, so in this part, we first acquire all the first- and second-order influence functions together based on the original model. After all the influence functions are calculated, we retrain the model corresponding to different removed coherent subset of data and get the ground truth.

In [24]:
time_gt = []
time_first = []
time_second = []
rep = 1

In [25]:
alpha_f_lower = (-0.01) * (spd_0)
alpha_f_upper = -spd_0
del_f_threshold = (0.1) * spd_0
support = 0.01 # Do not consider extremely small patterns
support_small = 0.3 # For small patterns, 2nd-order estimation is quite accurate
del_f_threshold_small = (-0.1) * (spd_0)
print("alpha_f_lower:", alpha_f_lower)
print("alpha_f_upper:", alpha_f_upper)
print("del_f_threshold:", del_f_threshold)
print("support_small:", support_small)
print("del_f_threshold_small:", del_f_threshold_small)

alpha_f_lower: 0.003634165891756614
alpha_f_upper: 0.3634165891756614
del_f_threshold: -0.03634165891756614
support_small: 0.3
del_f_threshold_small: 0.03634165891756614


In [26]:
for _ in range(rep):
    # Get the original model
#     clf = NeuralNetwork(input_size=X_train.shape[-1])
    clf = LogisticRegression(input_size=X_train.shape[-1])
    # clf = SVM()
    clf.fit(X_train, y_train)

    attributes = []
    attributeValues = []
    first_order_influences = []
    second_order_influences = []
    fractionRows = []

    # print("Attribute, Value, Ground-truth subset, Add 1st-order inf individual, \
    # Second-order subset influence, %rowsRemoved, Accuracy")
    # clf.fit(X_train, y_train)
    # continuous_cols = ['duration', 'credit_amt', 'install_rate', 'num_credits', 'residence']
    v1_orig = v1
    for col in X_train_orig.columns:
        if dataset == 'german':
            if "purpose" in col or "housing" in col: #dummy variables purpose=0 doesn't make sense
                vals = [1]
            else:
                vals = X_train_orig[col].unique()
        elif dataset == 'adult':
            vals = X_train_orig[col].unique()
        elif dataset == 'compas':
            vals = X_train_orig[col].unique()
        elif dataset == 'salary':
            vals = X_train_orig[col].unique()
        else:
            raise NotImplementedError
        for val in vals:
            print(col, val, sep=": ")
            idx = X_train_orig[X_train_orig[col] == val].index  
            if (len(idx)/len(X_train) > support):  
                X = np.delete(X_train, idx, 0)
                y = y_train.drop(index=idx, inplace=False)
                if len(y.unique()) > 1:
                    idx = X_train_orig[X_train_orig[col] == val].index 

                    # First-order subset influence
                    t0 = time.time()
                    del_f_1 = 0            
        #             for i in range(len(idx)):
        #                 del_f_1 += infs_1[idx[i]]
                   
#                     params_f_1 = first_order_group_influence(idx, del_L_del_theta)
#                     del_f_1 = np.dot(v1.transpose(), params_f_1)
#                     time_first.append(time.time()-t0)

                    # Second-order subset influence
                    t0 = time.time()
#                     print(col, val)
#                     params_f_2 = second_order_influence(clf, X_train, y_train, idx, del_L_del_theta)
#                     print(col, val)
                    params_f_2 = second_order_group_influence(idx, del_L_del_theta)
                    del_f_2 = np.dot(v1.transpose(), params_f_2)
                    time_second.append(time.time()-t0)

                    attributes.append(col)
                    attributeValues.append(val)
                    first_order_influences.append(del_f_1)
                    second_order_influences.append(del_f_2)
            #         gt_influences.append(inf_gt)
                    fractionRows.append(len(idx)/len(X_train)*100)

            #         print(col, val, inf_gt, del_f_1, del_f_2, len(idx)/len(X_train), accuracy, sep=", ")

sex: 0
sex: 1
rank: 3
rank: 2
rank: 1
year: 1
year: 0
year: 2
year: 3
degree: 1
degree: 0
y_degree: 2
y_degree: 0
y_degree: 1


In [27]:
# for _ in range(rep):
#     gt_influences = []
# #     v1s = []
#     for col in X_train_orig.columns:
#         if dataset == 'german':
#             if "purpose" in col or "housing" in col: #dummy variables purpose=0 doesn't make sense
#                 vals = [1]
#             else:
#                 vals = X_train_orig[col].unique()
#         elif dataset == 'adult':
#             continuous_cols = ['age', 'education.num', 'hours',]
#             if col in continuous_cols:
#                 vals = X_train_orig[col].unique()
#             else:
#                 vals = [1]
#         elif dataset == 'compas':
#             vals = X_train_orig[col].unique()
#         else:
#             raise NotImplementedError
#         for val in vals:
#             print(col, val)
#             idx = X_train_orig[X_train_orig[col] == val].index 
#             X = np.delete(X_train, idx, 0)
#             y = y_train.drop(index=idx, inplace=False)
#             inf_gt = 0
#             if len(y.unique()) > 1:
#                 # Ground truth subset influence
#                 t0 = time.time()
#                 clf.fit(np.array(X), np.array(y))
#                 y_pred = clf.predict_proba(np.array(X_test))
#                 if metric == 0:
#                     inf_gt = computeFairness(y_pred, X_test_orig, y_test, 0, dataset) - spd_0
#                 elif metric == 1:
#                     inf_gt = computeFairness(y_pred, X_test_orig, y_test, 1, dataset) - tpr_parity_0
#                 elif metric == 2:
#                     inf_gt = computeFairness(y_pred, X_test_orig, y_test, 2, dataset) - predictive_parity_0
#                 time_gt.append(time.time()-t0)
#                 accuracy = computeAccuracy(y_test, y_pred)
#                 gt_influences.append(inf_gt)
# #                 v1s.append(del_spd_del_theta(clf, X_test_orig, X_test, dataset))  # here, v1 corresponds to the metric spd

In [28]:
# def compute_cosine_theta(vec1: np.ndarray, vec2: np.ndarray):
#     """
#         Compute cosine of the angle between two vectors 
#     """
#     vec1 = vec1.squeeze()
#     vec2 = vec2.squeeze()
#     assert vec1.shape == vec2.shape
#     vec1 /= np.sqrt(np.sum(vec1**2))
#     vec2 /= np.sqrt(np.sum(vec2**2))
#     return np.dot(vec1.T, vec2)

In [29]:
# def compute_theta(vec1: np.ndarray, vec2: np.ndarray, measure='radian'):
#     """
#         Compute the angle between two vectors (angle/radian measure)
#     """
#     cos = compute_cosine_theta(vec1, vec2)
#     theta = np.arccos(cos)
#     if measure == 'angle':
#         theta = theta / np.pi * 180
#     return theta

In [30]:
# cos_thetas = []
# thetas = []
# grad_change_scales = []
# for vec in v1s:
#     cos_thetas.append(compute_cosine_theta(v1_orig, vec))
#     thetas.append(compute_theta(v1_orig, vec, measure='angle'))
#     grad_change_scales.append(np.sqrt(np.sum((vec-v1_orig)**2)))

In [31]:
# expl = [attributes, attributeValues, first_order_influences, second_order_influences, gt_influences, fractionRows, thetas, grad_change_scales]
# expl = (np.array(expl).T).tolist()

# explanations = pd.DataFrame(expl, columns=["attributes", "attributeValues", "first_order_influences", "second_order_influences", "gt_influences", "fractionRows", "gradient_angles", "grad_change_scales"])

expl = [attributes, attributeValues, first_order_influences, second_order_influences, fractionRows]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["attributes", "attributeValues", "first_order_influences", "second_order_influences", "fractionRows"])
explanations['second_order_influences'] = explanations['second_order_influences'].astype(float)
explanations['first_order_influences'] = explanations['first_order_influences'].astype(float)
# explanations['gt_influences'] = explanations['gt_influences'].astype(float)
explanations['fractionRows'] = explanations['fractionRows'].astype(float)

In [32]:
candidates = explanations[(explanations["second_order_influences"] > alpha_f_lower) 
                           & (explanations["second_order_influences"] < alpha_f_upper)]
candidates = copy.deepcopy(explanations)
candidates.loc[:, 'score'] = candidates.loc[:, 'second_order_influences']*100/candidates.loc[:, 'fractionRows']
# display(candidates)

In [33]:
len(candidates)

14

In [34]:
%%time
candidates_all = []

# Generating 1-candidates
candidates_1 = []
for i in range(len(candidates)):
    candidate = []
    candidate_i = candidates.iloc[i]
#     if ((candidate_i["second_order_influences"] > del_f_threshold) & 
#         (candidate_i["fractionRows"] > support)):
    if ((candidate_i["fractionRows"] > support_small) or
       ((candidate_i["fractionRows"] > support) & (candidate_i["second_order_influences"] > del_f_threshold))
       ):
        attr_i = candidate_i["attributes"]
        val_i = str(candidate_i["attributeValues"])
        predicates = []
        predicates.insert(0, attr_i + '=' + str(val_i))
        candidate.insert(0, predicates)
        candidate.insert(1, candidate_i["fractionRows"])
        candidate.insert(2, candidate_i["score"])
        candidate.insert(3, candidate_i["second_order_influences"])
        candidates_1.insert(i, candidate)
print("Generated: ", len(candidates_1), " 1-candidates")
candidates_1.sort()
# display(candidates_1)

for i in range(len(candidates_1)):
    if (float(candidates_1[i][2]) > support): # if score > top-k, keep in candidates, not otherwise
        candidates_all.insert(len(candidates_all), candidates_1[i])
    
# Generating 2-candidates
candidates_2 = []
for i in range(len(candidates_1)):
    attr_i = candidates_1[i][0][0].split("=")[0]
    val_i = int(candidates_1[i][0][0].split("=")[1])
    for j in range(i):
        # merge two candidates
        attr_j = candidates_1[j][0][0].split("=")[0]
        val_j = int(candidates_1[j][0][0].split("=")[1])
        
        if (attr_i != attr_j):
            idx = X_train_orig[(X_train_orig[attr_i] == val_i) &
                              (X_train_orig[attr_j] == val_j)].index 
            
            idx_i = X_train_orig[(X_train_orig[attr_i] == val_i)].index 
            idx_j = X_train_orig[(X_train_orig[attr_j] == val_j)].index 
            fractionRows = len(idx)/len(X_train) * 100
            isCompact = True
            if (len(idx) == min(len(idx_i), len(idx_j))): # pattern is not compact if intersection equals one of its parents
                isCompact = False
            if (fractionRows/100 > support):
                X = np.delete(X_train, idx, 0)
                y = y_train.drop(index=idx, inplace=False)

                size_hvp = 1
                params_f_2 = second_order_group_influence(idx, del_L_del_theta)
                del_f_2 = np.dot(v1.transpose(), params_f_2)
                    
#                 params_f_2 = second_order_influence(X_train, idx, size_hvp, del_L_del_theta, hessian_all_points)
#                 del_f_2 = np.dot(v1.transpose(), params_f_2)[0][0]
                
                score = del_f_2 * 100/fractionRows
                if (((score > candidates_1[i][2]) & (score > candidates_1[j][2]) 
#                      & (del_f_2 > del_f_threshold_small)
                    )
                   or (fractionRows/100 > support_small)):
                        candidate = []
                        predicates = []
                        predicates.insert(0, attr_i + '=' + str(val_i))
                        predicates.insert(1, attr_j + '=' + str(val_j))
                        candidate.insert(0, sorted(predicates, key=itemgetter(0)))                        
                        candidate.insert(1, len(idx)*100/len(X_train))
                        candidate.insert(2, score)
                        candidate.insert(3, del_f_2)
            #             print(candidate)
                        candidates_2.insert(len(candidates_2), candidate)  
                        if (isCompact):
                            candidates_all.insert(len(candidates_all), candidate)
print("Generated: ", len(candidates_2), " 2-candidates")
candidates_2.sort()

# Recursively generating the rest
candidates_L_1 = copy.deepcopy(candidates_2)
iter=2
while((len(candidates_L_1) > 0) & (iter < 4)):
    print("Generated: ", iter)    
    candidates_L = []
    for i in range(len(candidates_L_1)):
        candidate_i = candidates_L_1[i][0]
        for j in range(i):
            candidate_j = candidates_L_1[j][0]
            # if L-1 lists intersect
            intersect_candidates = set(candidate_i).intersection(candidate_j)
            if (len(intersect_candidates) == iter - 1):
                setminus_i = list(set(candidate_i) - intersect_candidates)[0].split("=")
                setminus_j = list(set(candidate_j) - intersect_candidates)[0].split("=")
                attr_i = setminus_i[0]
                val_i = int(setminus_i[1])
                attr_j = setminus_j[0]
                val_j = int(setminus_j[1])
                if (attr_i != attr_j):
                    # merge to get L list
                    merged_candidate = list(set(candidate_i + candidate_j))

                    idx_i_j = pd.Index(list(range(len(X_train_orig))))
                    for k in range(len(intersect_candidates)):
                        attr = list(intersect_candidates)[k].split("=")[0]
                        val = int(list(intersect_candidates)[k].split("=")[1])
                        idx_i_j = idx_i_j.intersection(X_train_orig[(X_train_orig[attr] == val)].index)
                    
                    idx_i = idx_i_j.intersection(X_train_orig[(X_train_orig[attr_i] == val_i)].index)
                    idx_j = idx_i_j.intersection(X_train_orig[(X_train_orig[attr_j] == val_j)].index)                    
                    idx = idx_i.intersection(X_train_orig[(X_train_orig[attr_j] == val_j)].index) # merged

                    fractionRows = len(idx)/len(X_train) * 100
                    isCompact = True
                    if (len(idx) == min(len(idx_i), len(idx_j))): # pattern is not compact if intersection equals one of its parents
                        isCompact = False
                    if (fractionRows/100 > support):
                        X = np.delete(X_train, idx, 0)
                        y = y_train.drop(index=idx, inplace=False)

                        size_hvp = 1
                        params_f_2 = second_order_group_influence(idx, del_L_del_theta)
                        del_f_2 = np.dot(v1.transpose(), params_f_2)
                    
#                         params_f_2 = second_order_influence(X_train, idx, size_hvp, del_L_del_theta, hessian_all_points)
#                         del_f_2 = np.dot(v1.transpose(), params_f_2)[0][0]
    
                        score = del_f_2 * 100/fractionRows
                        if (((score > candidates_L_1[i][2]) & (score > candidates_L_1[j][2]) 
#                              & (del_f_2 > del_f_threshold_small)
                            ) or 
                           (fractionRows > support_small)):
                            candidate = []
                            candidate.insert(0, sorted(merged_candidate, key=itemgetter(0)))                        
                            candidate.insert(1, fractionRows)
                            candidate.insert(2, del_f_2*len(X_train)/len(idx))
                            candidate.insert(3, del_f_2)
                            if (candidate not in candidates_L):
                                candidates_L.insert(len(candidates_L), candidate)
                                if (isCompact):
                                    candidates_all.insert(len(candidates_all), candidate)
    print("Generated:", len(candidates_L), " ", str(iter+1), "-candidates")
    candidates_L_1 = copy.deepcopy(candidates_L)
    candidates_L_1.sort()
    iter += 1

Generated:  14  1-candidates
Generated:  20  2-candidates
Generated:  2
Generated: 23   3 -candidates
Generated:  3
Generated: 17   4 -candidates
CPU times: user 551 ms, sys: 20.4 ms, total: 571 ms
Wall time: 572 ms


In [35]:
candidates_support_3_compact = copy.deepcopy(candidates_all)
print(len(candidates_support_3_compact))
candidates_df_3_compact = pd.DataFrame(candidates_support_3_compact, columns=["predicates","support","score","2nd-inf"])
# candidates_df_3_compact = candidates_df_3_compact[candidates_df_3_compact["support"] < 10].sort_values(by=['score'], ascending=False)
print(len(candidates_df_3_compact))
# display(candidates_df_3_compact)
# display(candidates_df_3_compact[candidates_df_3_compact["support"] < 20].sort_values(by=['2nd-inf'], ascending=False).head(5))

50
50


**Containment-based filtering (with LSH Ensemble)**

In [36]:
def get_subset(X_train_orig, explanation):
    subset = X_train_orig.copy()
    for predicate in explanation:
#         print(predicate)
        attr = predicate.split("=")[0].strip(' ')
        val = int(predicate.split("=")[1].strip(' '))
        subset = subset[subset[attr]==val]
    return subset.index

from datasketch import MinHashLSHEnsemble, MinHash, MinHashLSH
import json

class Topk:
    '''
        top explanations: explanation -> (minhash, set_index, score)
    '''
    def __init__(self, method='lsh', init_df=X_test_orig, init_explanations=[], threshold=0.75, k=10, num_perm=128):
        self.method = method
        self.num_perm = num_perm
        if method == 'lshensemble':
#             self.index = MinHashLSHEnsemble(threshold=threshold, num_perm=num_perm)
#             hashed_explanations = []
#             for explanation, score in init_explanations:
#                 s = get_subset(init_df, explanation)
#                 m = MinHash(num_perm=num_perm)
#                 explanation_json = json.dumps(explanation)
#                 for d in s:
#                     m.update(str(d).encode('utf8'))
#                 hashed_explanations.append((explanation_json, m, len(s)))
#             self.index.index(hashed_explanations)
            raise NotImplementedError
        elif method == 'lsh':
#             self.index = MinHashLSH(threshold=threshold, num_perm=num_perm, params=[64, self.num_perm//64])
#             for explanation, score in init_explanations:
#                 s = get_subset(init_df, explanation)
#                 m = MinHash(num_perm=num_perm)
#                 explanation_json = json.dumps(explanation)
#                 for d in s:
#                     m.update(str(d).encode('utf8'))
#                 self.index.insert(explanation_json, m)
            raise NotImplementedError

        self.top_explanations = dict()
        self.k = k
        self.threshold = threshold
        self.min_score = -100
        self.min_score_explanation =None
    
    def _update_min(self, new_explanation, new_score):
        if len(self.top_explanations) > 0:
            for explanation, t in self.top_explanations.items():
                if t[2] < new_score:
                    new_score = t[2]
                    new_explanation = explanation
        self.min_score = new_score
        self.min_score_explanation = new_explanation
        
    def _containment(self, x, q):
        return len(x & q)/len(q)
            
    def update(self, df, explanation, score):
        if (len(self.top_explanations) < self.k) or (score > self.min_score):
            s = get_subset(df, explanation)
            m = MinHash(num_perm=self.num_perm)
            explanation = json.dumps(explanation)
            for d in s:
                m.update(str(d).encode('utf8'))

            if self.method == 'lshensemble':
#                 q_result = set(self.index.query(m, len(s))).intersection(set(self.top_explanations.keys()))
                raise NotImplementedError
            elif self.method == 'lsh':
#                 q_result = set(self.index.query(m)).intersection(set(self.top_explanations.keys()))
                raise NotImplementedError
            elif self.method == 'containment':
                q_result = set()
                for k, v in self.top_explanations.items():
                    if self._containment(v[1], s) > self.threshold:
#                         print(self._containment(v[1], s))
                        q_result.add(k)
            
#             print(q_result)
#             print([round(len(self.top_explanations[q][1] & s)/len(s), 3) for q in q_result])
#             print([round(len(self.top_explanations[q][1] & s)/len(self.top_explanations[q][1] | s), 3) for q in self.top_explanations.keys()])

            if len(q_result)==0:
                if len(self.top_explanations) <= self.k-1:
                    self._update_min(explanation, score)
                    self.top_explanations[explanation] = (m, s, score)
                    return 0
#                 else:
#                     del self.top_explanations[self.min_score_explanation]
#                     self._update_min(explanation, score)
#                     self.top_explanations[explanation] = (m, s, score)
#                     return 0
#             else:
#                 q_scores = [self.top_explanations[explanation][2] for explanation in q_result]
#                 if max(q_scores) < score:
#                     for explanation in q_result:
#                         del self.top_explanations[explanation]
#                     self._update_min(explanation, score)
#                     self.top_explanations[explanation] = (m, s, score)
# #                     print('Inserted')
#                     return 0
        return -1

lsh_df = candidates_df_3_compact.sort_values(by=['score'], ascending=False).copy()

topk = Topk(method='containment', threshold=0.2, k=10)
for row_idx in range(len(lsh_df)):
    row = lsh_df.iloc[row_idx]
    explanation, score = row[0], row[2]
    topk.update(X_train_orig, explanation, score)
    if len(topk.top_explanations) == topk.k:
        break

In [37]:
explanations = list(topk.top_explanations.keys())
idxs = [v[1] for v in topk.top_explanations.values()]
supports = list()
scores = list()
infs = list()
for e in explanations:
    condition = candidates_df_3_compact.predicates.apply(lambda x: x==json.loads(e))
    supports.append(float(candidates_df_3_compact[condition]['support']))
    scores.append(float(candidates_df_3_compact[condition]['score']))
    infs.append(float(candidates_df_3_compact[condition]['2nd-inf']))

expl = [explanations, supports, scores, infs]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["explanations", "support", "score", "2nd-inf"])
explanations['score'] = explanations['score'].astype(float)
explanations['support'] = explanations['support'].astype(float)
explanations['2nd-inf'] = explanations['2nd-inf'].astype(float)/(spd_0)

pd.set_option('max_colwidth', 100)
explanations.sort_values(by=['score'], ascending=False)

,explanations,support,score,2nd-inf
0,"[""rank=2"", ""sex=0""]",2.439024,2.592719,-0.174007
1,"[""rank=2"", ""year=1""]",12.195122,1.222780,-0.410327
2,"[""degree=1"", ""rank=2"", ""year=2""]",4.878049,1.128779,-0.151513
3,"[""degree=0"", ""y_degree=0""]",2.439024,0.821300,-0.055120
4,"[""degree=1"", ""y_degree=1""]",26.829268,0.579219,-0.427609
5,"[""degree=0"", ""rank=2"", ""year=2""]",2.439024,0.528921,-0.035498
6,"[""rank=1"", ""sex=0""]",17.073171,0.305384,-0.143468
7,"[""rank=1"", ""year=3""]",2.439024,0.140269,-0.009414
8,"[""year=3""]",12.195122,0.079724,-0.026753
9,"[""degree=1"", ""sex=1"", ""year=0""]",14.634146,-0.717288,0.288839


In [38]:
idx = X_train_orig[
                   (X_train_orig['rank']==2)
                   &(X_train_orig['year']==1)
#                    & (X_train_orig['degree']==1)
#                    & (X_train_orig['foreign_worker']==1)
            ].index 

print("%rows", len(idx)*100/len(X_train))
X = np.delete(X_train, idx, 0)
y = y_train.drop(index=idx, inplace=False)
clf.fit(X, y)
y_pred = clf.predict_proba(X_test)
print("inf-gt", computeFairness(y_pred, X_test_orig, y_test, 0, dataset)/spd_0 - 1)
# print("gt", computeFairness(y_pred, X_test_orig, y_test, 0, dataset))

params_f_2 = second_order_group_influence(idx, del_L_del_theta)
del_f_2 = np.dot(v1.transpose(), params_f_2)
print("inf-2", del_f_2/spd_0)

print(del_f_2/(len(idx)/len(X_train)))

%rows 12.195121951219512
inf-gt -0.7511298325530031
inf-2 -0.4103266675566325
1.2227800473642245


In [39]:
df = copy.deepcopy(X_train_orig)
df['salary'] = list(y_train)
df.iloc[idx]

,sex,rank,year,degree,y_degree,salary
1,1,2,1,0,2,0
3,1,2,1,1,1,1
8,0,2,1,0,2,0
10,1,2,1,0,2,0
24,1,2,1,0,1,0


In [40]:
df[(df['rank']==2)]

,sex,rank,year,degree,y_degree,salary
1,1,2,1,0,2,0
3,1,2,1,1,1,1
8,0,2,1,0,2,0
10,1,2,1,0,2,0
12,1,2,2,1,1,1
13,1,2,2,0,1,1
18,1,2,0,1,0,0
20,1,2,0,0,1,1
21,1,2,0,0,0,1
24,1,2,1,0,1,0


100%|██████████| 41/41 [00:20<00:00, 165.83it/s]

In [ ]:
len(df[df['sex'] == 1])

In [ ]:
rank=1

len(df[(df['year']>1) & (df['year']<=3) & (df['sex']==0)])/len(df[(df['year']>1) & (df['year']<=3)])

In [ ]:
df[["sex", ]].groupby(by="sex", dropna=False).sum()

In [ ]:
print(computeFairness(y_pred, X_test_orig, y_test, 0, dataset))
spd_0

In [ ]:
explanations = list(topk.top_explanations.keys())
idxs = [v[1] for v in topk.top_explanations.values()]
supports = list()
scores = list()
infs = list()
for e in explanations:
    condition = candidates_df_3_compact.predicates.apply(lambda x: x==json.loads(e))
    supports.append(float(candidates_df_3_compact[condition]['support']))
    scores.append(float(candidates_df_3_compact[condition]['score']))
    infs.append(float(candidates_df_3_compact[condition]['2nd-inf']))

expl = [explanations, supports, scores, infs]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["explanations", "support", "score", "2nd-inf"])
explanations['score'] = explanations['score'].astype(float)
explanations['support'] = explanations['support'].astype(float)
explanations['2nd-inf'] = explanations['2nd-inf'].astype(float)/(-spd_0)

pd.set_option('max_colwidth', 100)
explanations.sort_values(by=['score'], ascending=False)

In [ ]:
lim = 0.1
plt.figure(figsize=(5,5))
xs = (np.arange(31)-15)/15*lim*0.8
ys = xs
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)
plt.plot(xs, ys, 'grey')

# RGBA
color_first = np.zeros((len(gt_influences), 4))
color_first[:, 2] = 1.0
color_first[:, 3] = 1-np.array(fractionRows)/100
color_second = np.zeros((len(gt_influences), 4))
color_second[:, 0] = 1.0
color_second[:, 3] = 1-np.array(fractionRows)/100

# color_first = np.zeros((len(gt_influences), 4))
# color_first[:, 0] = 1.0
# color_first[:, 3] = 0.2
# color_second = np.zeros((len(gt_influences), 4))
# color_second[:, 0] = 1.0
# color_second[:, 3] = 1.0

plt.scatter(gt_influences, first_order_influences, s=12, color=color_first, label='first-order')
plt.scatter(gt_influences, second_order_influences, s=12, color=color_second, label='second-order')
plt.ylabel('estimated influence\n', fontsize=12, fontweight='bold')
plt.xlabel('\nground truth influence', fontsize=12, fontweight='bold')
plt.legend(fontsize=12, prop={'weight':'bold'})
plt.grid()
plt.savefig('infs.png')

In [ ]:
time_gt_ave = []
time_second_ave = []
time_first_ave = []

l = len(time_gt)//rep
for i in range(l):
    time_gt_ave.append(np.average([time_gt[i+j*l] for j in range(rep)]))
    time_second_ave.append(np.average([time_second[i+j*l] for j in range(rep)]))
    time_first_ave.append(np.average([time_first[i+j*l] for j in range(rep)]))

In [ ]:
plt.figure(figsize=(5,5))
# plt.subplot(131)
sorted_idx = explanations.sort_values(by=['fractionRows'], ascending=True).index
xs = []
ys = []
for idx in sorted_idx:
    xs.append(fractionRows[idx])
    ys.append(time_first_ave[idx])
plt.plot(xs, ys, '-', c='blue', label='first-order')

# plt.subplot(132)
xs = []
ys = []
for idx in sorted_idx:
    xs.append(fractionRows[idx])
    ys.append(time_second_ave[idx])
plt.plot(xs, ys, '-', c='red', label='second-order')

# plt.subplot(133)
xs = []
ys = []
for idx in sorted_idx:
    xs.append(fractionRows[idx])
    ys.append(time_gt_ave[idx])
plt.plot(xs, ys, '-', c='green', label='ground truth')
plt.legend(fontsize=12, prop={'weight':'bold'})
plt.grid()
plt.xlabel('\nfraction of data removed (%)', fontsize=12, fontweight='bold')
plt.ylabel('time cost / sec\n', fontsize=12, fontweight='bold')
# plt.show()
plt.savefig('time.png')

In [ ]:
# bucket_num = 10
fractionRows = np.array(fractionRows)
gt_influences = np.array(gt_influences)
first_order_influences = np.array(first_order_influences)
second_order_influences = np.array(second_order_influences)
corr_first_gt_ls = []
corr_second_gt_ls = []
for bucket_id in range(10):
    is_in_bucket = np.logical_and(fractionRows>=bucket_id*10, fractionRows<(bucket_id+1)*10)
    corr_first_gt = np.corrcoef([gt_influences[is_in_bucket], first_order_influences[is_in_bucket]])[0][1]
    if np.isnan(corr_first_gt):
        corr_first_gt = 0
    corr_second_gt = np.corrcoef([gt_influences[is_in_bucket], second_order_influences[is_in_bucket]])[0][1]
    if np.isnan(corr_second_gt):
        corr_second_gt = 0
    corr_first_gt_ls.append(corr_first_gt)
    corr_second_gt_ls.append(corr_second_gt)

In [ ]:
plt.figure(figsize=(8,8))
bar_width=0.3
tick_label=[f'{i*10}%-{(i+1)*10}%' for i in range(10)]
plt.bar(np.arange(10), corr_first_gt_ls, bar_width, color='blue', label='first')
plt.bar(np.arange(10)+bar_width, corr_second_gt_ls, bar_width, color='orange', label='second')
plt.legend(fontsize=12)
plt.xticks(np.arange(10)+bar_width/2, tick_label, rotation=20, fontsize=12)
plt.xlabel('Row Fraction', fontsize=20)
plt.ylabel('Pearson Correlation', fontsize=20)
plt.show()

In [ ]:
# overall correlation
np.corrcoef([gt_influences, first_order_influences])[0][1], np.corrcoef([gt_influences, second_order_influences])[0][1]

less than 2 coherent subsets correspond to the bracket 50%-60%, therefore cannot compute corrsponding pearson correlation

In [ ]:
# lim = 0.01
plt.figure(figsize=(15, 6))
for bucket_id in range(10):
    plt.subplot(2, 5, bucket_id+1)
    is_in_bucket = np.logical_and(fractionRows>=bucket_id*10, fractionRows<(bucket_id+1)*10)
    xs = (np.arange(30)-15)/15*lim
    ys = xs
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.plot(xs, ys, 'g')
    plt.scatter(gt_influences[is_in_bucket], first_order_influences[is_in_bucket], s=10, c='blue', label='first')
    plt.scatter(gt_influences[is_in_bucket], second_order_influences[is_in_bucket], s=10, c='orange', label='second')
    plt.legend()
    plt.title(f'Fraction: {bucket_id*10}%-{(bucket_id+1)*10}% {np.sum(is_in_bucket)}')
plt.tight_layout()
plt.show()

For buckets which include only small num of coherent subsets (many of them corresponds to high row fraction), the pearson correlation can be less meaningful. However, from the scatter plots above, we can see that the estimation of influence function tends to be more accurate when the row fractions are relatively low, meanwhile, low row fraction leads to low overall influence (change of metrics caused by removing the subset) of coherent subsets.